# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [28]:
import numpy as np
from sklearn.datasets import load_iris

def build_kernel(data_set, kernel_type='linear',degree=3):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = np.dot(data_set, data_set.T)
        kernel = np.power(kernel, degree)
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [29]:
def choose_set_for_label(data_set, labels, classes):
    mask = np.isin(labels, classes)
    filtered_data = data_set[mask]
    filtered_labels = labels[mask]
    train_labels = np.where(filtered_labels == classes[0], -1, 1)

    return filtered_data, train_labels

In [10]:
def get_labels_count(data_set):
    return len(np.unique(data_set))

Use the code that we have implemented earlier:

In [30]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)
    objects_count = len(train_data_set)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [31]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt
from itertools import combinations
from sklearn.metrics import accuracy_score

iris = load_iris()
data_set = iris.data
labels = iris.target

train_data_set, test_data_set, train_labels, test_labels = train_test_split(
    data_set, labels, test_size=0.4, random_state=16
)

labels_combinations = list(combinations(range(get_labels_count(labels)), 2))

votes = np.zeros((len(test_data_set), get_labels_count(labels)))

for classes in labels_combinations:
    filtered_data, binary_labels = choose_set_for_label(train_data_set, train_labels, classes)
    
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(
        filtered_data, binary_labels, kernel_type='rbf'
    )
    
    predicted_labels = classify_rbf(
        test_data_set, filtered_data, lambdas, targets, b, vector_number, support_vectors, support_vectors_id
    )
    
    assigned_classes = np.where(predicted_labels.flatten() == -1, classes[0], classes[1])

    for idx, cls in enumerate(assigned_classes):
        votes[idx, cls] += 1

final_predictions = np.argmax(votes, axis=1)

accuracy = accuracy_score(test_labels, final_predictions)
print(f"Accuracy: {accuracy:.4f}")


     pcost       dcost       gap    pres   dres
 0:  9.0982e+01 -9.5636e+02  2e+03  1e-01  2e-15
 1:  4.7786e+01 -9.4909e+01  2e+02  5e-03  2e-15
 2:  5.1544e+00 -1.3650e+01  2e+01  7e-16  2e-15
 3: -7.1636e-01 -3.2627e+00  3e+00  2e-16  8e-16
 4: -1.1914e+00 -1.7466e+00  6e-01  2e-16  3e-16
 5: -1.3686e+00 -1.5661e+00  2e-01  2e-16  2e-16
 6: -1.4291e+00 -1.4833e+00  5e-02  2e-16  2e-16
 7: -1.4542e+00 -1.4601e+00  6e-03  2e-16  2e-16
 8: -1.4564e+00 -1.4565e+00  1e-04  2e-16  2e-16
 9: -1.4564e+00 -1.4564e+00  2e-06  2e-16  2e-16
10: -1.4564e+00 -1.4564e+00  2e-08  2e-16  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.0498e+02 -1.0701e+03  2e+03  1e-01  2e-15
 1:  5.3584e+01 -1.0896e+02  2e+02  5e-03  2e-15
 2:  5.7517e+00 -1.5619e+01  2e+01  4e-16  2e-15
 3: -8.8834e-01 -3.7649e+00  3e+00  2e-16  8e-16
 4: -1.4296e+00 -2.1699e+00  7e-01  1e-16  3e-16
 5: -1.6392e+00 -1.9091e+00  3e-01  2e-16  2e-16
 6: -1.7256e+00 -1.8093e+00  8e-02  2e-16  2e-1